In [17]:
# Imports
from pathlib import Path
from massimal import georeferencing, hyspec_io
import warnings
from tqdm import tqdm

In [18]:
# Paths
rrs_dir = Path('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs')
geotiff_dir = rrs_dir.parent / '2d_R_rs_RGB_GeoTiff'
world_file_dir = geotiff_dir
geotiff_dir.mkdir(exist_ok = True)


In [19]:
hyspec_paths = sorted(rrs_dir.glob('*.hdr'))
print(hyspec_paths)

[PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_1-Crop Wavelengths.bip.hdr'), PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_10-Crop Wavelengths.bip.hdr'), PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_11-Crop Wavelengths.bip.hdr'), PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_12-Crop Wavelengths.bip.hdr'), PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_13-Crop Wavelengths.bip.hdr'), PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_14-Crop Wavelengths.bip.hdr'), PosixPath('/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_15-Crop Waveleng

In [21]:
for hyspec_path in tqdm(hyspec_paths): 
    hyspec_path_stem = hyspec_path.stem.split('.')[0]
    lcf_path = hyspec_path.parent / (hyspec_path_stem + '.lcf')
    times_path = hyspec_path.parent / (hyspec_path_stem + '.bip.times')
    world_file_path = world_file_dir / (hyspec_path_stem + '.wld')
    geotiff_path = geotiff_dir / (hyspec_path_stem + '.tiff')
    
    print(hyspec_path_stem)
    print(lcf_path)
    print(times_path)
    print(world_file_path)
    print(geotiff_path)

    if lcf_path.exists() and times_path.exists():
        utm_epsg = georeferencing.world_file_from_lcf_times_files(lcf_path,times_path,world_file_path)
        (image,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(hyspec_path,rgb_only=True)

        georeferencing.save_geotiff_with_affine_transform(
            image=image,
            crs_str=f'EPSG:{utm_epsg}',
            geotiff_path=geotiff_path,
            world_file_path=world_file_path,
            band_names=tuple([str(wl[band_ind]) for band_ind in rgb_ind])
        )
    else:
        warnings.warn(f'{hyspec_path_stem}: Missing .lcf or .times file')

  0%|          | 0/36 [00:00<?, ?it/s]

Juvika_June2022_Pika_L_1-Crop Wavelengths
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_1-Crop Wavelengths.lcf
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_1-Crop Wavelengths.bip.times
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_1-Crop Wavelengths.wld
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_1-Crop Wavelengths.tiff


  3%|▎         | 1/36 [00:07<04:30,  7.74s/it]

Juvika_June2022_Pika_L_10-Crop Wavelengths
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_10-Crop Wavelengths.lcf
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_10-Crop Wavelengths.bip.times
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_10-Crop Wavelengths.wld
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_10-Crop Wavelengths.tiff


  6%|▌         | 2/36 [00:15<04:14,  7.49s/it]

...

 94%|█████████▍| 34/36 [03:50<00:12,  6.33s/it]

Juvika_June2022_Pika_L_8-Crop Wavelengths
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_8-Crop Wavelengths.lcf
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_8-Crop Wavelengths.bip.times
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_8-Crop Wavelengths.wld
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_8-Crop Wavelengths.tiff


 97%|█████████▋| 35/36 [03:57<00:06,  6.70s/it]

Juvika_June2022_Pika_L_9-Crop Wavelengths
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_9-Crop Wavelengths.lcf
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs/Juvika_June2022_Pika_L_9-Crop Wavelengths.bip.times
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_9-Crop Wavelengths.wld
/media/mha114/Massimal2/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2d_R_rs_RGB_GeoTiff/Juvika_June2022_Pika_L_9-Crop Wavelengths.tiff


100%|██████████| 36/36 [04:05<00:00,  6.83s/it]
